In [ ]:
# Q2
def find_pairs(arr, target):
    arr.sort()
    left, right = 0, 1
    res = []
    while right < len(arr):
        diff = arr[right] - arr[left]
        
        # diff < target, means that we need a bigger arr[right]
        if diff < target:
            right += 1

        # diff > target, need a bigger arr[left]
        elif diff > target:
            left += 1
        
        elif diff == target:
            res.append((arr[left], arr[right]))
            right += 1      # start searching next pairs
        
        if left == right:
            right += 1      # prevent from left == right

    return res if res else "False"

In [4]:
# 测试
arr = [1, 5, 3, 7, 9, 11]
target = 4
print(find_pairs(arr, target))  # 输出 [(1, 5), (3, 7), (5, 9), (7, 11)]


[(1, 5), (3, 7), (5, 9), (7, 11)]


In [ ]:
# Q3
def skyline(arr: list[int]):
    class stack():
        def __init__(self,value = [],arr = []):
            self.value = value
            self.arr = arr
        
        def isEmpty(self):
            return len(self.value) == 0 

        def top(self):
            return self.value[-1] if not self.isEmpty() else None

        def pop(self):
            return self.value.pop()

        # The different from average stack is push
        def push(self,element):
            '''
            When the arr[top of the stack] is smaller than the arr[element]
            We pop that, until we find the bigger arr[top] or the Stack become empty
            '''
            while not self.isEmpty() and self.arr[self.top()] <= self.arr[element]:
                self.pop()
            res = self.top() # we return the previous top 
            self.value.append(element)
            return res
            

    def get_left(arr):
        res = []
        inorder_stack = stack([],arr)
        for i, value in enumerate(arr):
            res.append(inorder_stack.push(i))
        return res
    
    def get_right(arr):
        # get right is a reverse of get_left, also the result will be reversed
        res = get_left(arr[::-1])[::-1]
        for i in range (0,len(res)):
            # Finding that the index change with the reverse operation, we need to get it back
            if res[i] != None:
                res[i] = len(res) - res[i] - 1
        return res
            
    return get_left(arr), get_right(arr)

        



In [112]:
skyline([7,3,9,12,2,6,5,15])
skyline([6,2,4,1,10,7,8,11])

([None, 0, 0, 2, None, 4, 4, None], [4, 2, 4, 4, 7, 6, 7, None])

In [ ]:
# Q4
# Define that the first half always >=  second half(When even length, equal ; when odd length, > )
class Order():
    def __init__(self, order_id : str , value : float):
        self.order_id = order_id
        self.value = value
        self.next = None
        self.prev = None
        self.left_queue = False     # Every new order starting from right_queue
        

class Bob_queue():
    def __init__(self):
        self.status = True                                              # define the Bob is closed or not
        self.left_queue_head, self.right_queue_head = None, None        # define the head of the left_queue and right_queue
        self.left_queue_tail, self.right_queue_tail = None, None        # define the tail of both queue  
        self.left_queue_length, self.right_queue_length = 0, 0          # define length
        self.left_queue_sum, self.right_queue_sum = 0, 0                # define sum

    # check the length of right and left; then do operation to balance them
    def balance_check(self):
        # the right queue may be longer, so we need to put the right_queue[head] to the left_queue[tail]
        while self.left_queue_length < self.right_queue_length:
            # When the left queue is empty, we put it in the left[head]
            # in this situation, right at least has 1 element
            node = self.right_queue_head

            if self.left_queue_length == 0 :
                # put node to the left queue    
                node.prev = self.left_queue_tail
                self.left_queue_head = node 
                self.left_queue_tail = node
                node.left_queue = True

                # delete node from right queue
                self.right_queue_head = node.next

                # the right queue might be None
                if node.next == None:
                    self.right_queue_tail = None
                node.next = None
                
            # Normal case
            else:
                # put node to the left queue
                node.prev = self.left_queue_tail
                self.left_queue_tail.next = node
                self.left_queue_tail = node
                node.left_queue = True
                
                # delete node from right queue
                self.right_queue_head = node.next
                node.next = None    # don`t need to use if because left_queue always > 1 and right queue must > 2

            
            # generate length and sum
            self.right_queue_length -= 1
            self.right_queue_sum -= node.value
            self.left_queue_length += 1
            self.left_queue_sum += node.value
                



    def placeOrder(self, item: Order):
        if not self.status: 
            raise Exception
        
        # When the right tail is empty, we put it at right[head] while right[tail] is item too.
        if self.right_queue_length == 0:
            self.right_queue_head = item
            self.right_queue_tail = item
        
        # When the right tail is not empty, we put it at the right[tail]
        else:
            self.right_queue_tail.next = item
            self.right_queue_tail = item
        
        # generate sum and length of right queue
        self.right_queue_length += 1
        self.right_queue_sum += item.value

        # use balance_check to make sure that left length > right length
        self.balance_check()


    def serve(self):
        node = self.left_queue_head
        self.left_queue_sum -= node.value
        self.left_queue_length -= 1

        if node.next == None:
            self.left_queue_head = None
            self.left_queue_tail = None
        else:
            self.left_queue_head = node.next
            self.left_queue_head.prev = None
        node.next = None
        self.balance_check()
        return node


    def closeShop(self):
        self.status = False

    def openShop(self):
        self.status = True
    
    def busyNow(self):
        if self.left_queue_length + self.right_queue_length == 0:
            return False
        
        if self.left_queue_sum > self.right_queue_sum:
            return True
        
        else:
            return False

    def cancelOrder(self, item: Order):

        # if item in the left_queue
        if item.left_queue:  
            self.left_queue_sum -= item.value
            self.left_queue_length -= 1

            # item is the head
            if self.left_queue_head == item :
                self.left_queue_head = item.next
                if self.left_queue_head != None:
                    self.left_queue_head.prev = None

            # item is the tail
            if self.left_queue_tail == item :
                self.left_queue_tail = item.prev
                if self.left_queue_tail != None:
                    self.left_queue_tail.next = None

        # if item in the right_queue
        else:  
            self.right_queue_sum -= item.value
            self.right_queue_length -= 1

            # item is the head
            if self.right_queue_head == item :
                self.right_queue_head = item.next
                if self.right_queue_head != None:
                    self.right_queue_head.prev = None

            # item is the tail
            if self.right_queue_tail == item:
                self.right_queue_tail = item.prev
                if self.right_queue_tail:
                    self.right_queue_tail.next = None

       
        if item.prev != None:
            item.prev.next = item.next

        if item.next != None:
            item.next.prev = item.prev

        # remove item
        item.next = None
        item.prev = None
        del item
        self.balance_check()

